# Ipython of Real-time Minituar leg jumping with stairs analysis#
## Nov. 5, 2017##
This file is the data analysis of real-time data captured by Qualisys Camera, in 100 Hz. And Arduino board's signal is streamed at the same time. Amazingly, when the Arduino signal time is captured together with Camera's signal in the same loop, the updating rate becomes stabler, and the updating hertz keeps at 100Hz.

In [1]:
%matplotlib 

Using matplotlib backend: Qt5Agg


 to view the result, please read
    - 1 jump per stair changing:
        20171104-21_21_54-deadbeat-v0.1.0-jumpstair.csv;
        20171104-21_29_49-deadbeat-v0.1.0-jumpstair.csv;
    - 5 jump5 per stair changing:
        20171104-21_37_40-deadbeat-v0.1.0-jump5_stair.csv;
        20171104-21_41_58-deadbeat-v0.1.0-jump5_stair.csv;

In [22]:
import pandas as pd
import numpy as np
import read as rd
import matplotlib.pyplot as plt
from numpy import linalg as LA

df = pd.read_csv('20171104-21_37_40-deadbeat-v0.1.0-jump5_stair.csv')

## 1. Read .csv file##
.csv file is real-time data streamed in python and saved in Ubuntu VirtureMachine.

In [23]:
X = df.x;
Y = df.y;
T = df.time;
ARD = df.ard;
#x, y, t= rd.jump_from_csv(X, Y, T)

In [24]:
def jump_from_csv(X,Y,T,ARD):
    # This function is to read csv data#
    
    length = len(X);
    x,y,t,ard = np.zeros((length,5)), np.zeros((length,5)), np.zeros((length,1)),np.zeros((length,6));
    for i in range(length):
        x_line = X[i].replace("[","");
        y_line = Y[i].replace("[",""); 
        trans_x = x_line.replace("]","");
        trans_y = y_line.replace("]","");
        ard_line = ARD[i].replace("[",""); 
        trans_ard = ard_line.replace("]","");
        trans_ard = trans_ard.replace("\\n","");
        trans_ard = trans_ard.replace("'","");
        if (len(trans_x.split(" ")) == 4):
            for j in range(4):
                x[i,j] = float(trans_x.split(" ")[j])
                y[i,j] = float(trans_y.split(" ")[j])
                t[i] = float(T[i])
            for j in range(4):
                ard[i,j] = float(trans_ard.split(" ")[j])
        else:
            pass
               
    return x, y, t, ard
x, y, t, ard= jump_from_csv(X, Y, T,ARD)

In [25]:
x = x[~(x==0).all(1)];
y = y[~(y==0).all(1)];
t = t[~(t==0).all(1)];
ard = ard[~(ard==0).all(1)];
first = t[0];
t[:] = [i - first for i in t]
t = t / 1000

In [26]:
np.isclose(len(ard), len(x))

True

## 2. Data  classify##
Four points on the robot are marked and captured, which is 
- Marker on the top of robot: p1
- Marker on right knee: p2
- Marker on left knee: p3
- Marker on the foot: p4

In [27]:
def jumpstairs_coordinate(x, y, t):
    p1, p2, p3, p4, p5 = np.zeros((len(x),3)), np.zeros((len(x),3)), np.zeros((len(x),3)), np.zeros((len(x),3)),  np.zeros((len(x),3));
    for i in range(len(x)):
        po1_idx=np.argmax(x[i]);
        po2_idx=np.argmin(y[i]);
        po3_idx=np.argmin(x[i]);
        po4_idx=np.argmax(y[i]);
        po5_idx=[element for j, element in enumerate([0,1,2]) if j not in {po1_idx, po2_idx, po3_idx, po4_idx}];
        p1[i]=[x[i, po1_idx],y[i, po1_idx], t[i]];
        p2[i]=[x[i, po2_idx],y[i, po2_idx], t[i]];
        p3[i]=[x[i, po3_idx],y[i, po3_idx], t[i]];
        p4[i]=[x[i, po4_idx],y[i, po4_idx], t[i]];
        p5[i]=[x[i, po5_idx[0]],y[i, po5_idx[0]] , t[i]];
    return p1, p2, p3, p4, p5

p1, p2, p3, p4, p5 = jumpstairs_coordinate(x, y, t)# p1 is the top, p2 is right ness, p3 is left knee, p4 is foot, p5 is the joint above the foot

## 3. Calibration##
I used an 'L' shaped frame, which is the shape of Cartesian coordinate system, with three points to calbirate the camera
- P1: horizontal ('x') coordinate
- P2: vertical ('y') coordinate
- P3: Origin

In [28]:
P1 = np.array([ 61485.00233801,  61237.35337341]);
P2 = np.array([ 50265.03640615,  50470.93353373]);
P3 = np.array([ 50584.49331997,  61035.10888444]); # P3 is the origin of coordinator



top marker and foot marker after calibration

In [29]:
top_cal = rd.calibration(p1, P1, P2, P3)
foot_cal = rd.calibration(p4, P1, P2, P3)

In [30]:
#plt.title('trial one Minituar jumping real-time capture');
plt.figure(figsize=(14,14));
plt.title('trial one Minituar jumping real-time capture');
plt.plot(t,foot_cal[:,1],label="foot")
plt.plot(t,top_cal[:,1],label="top")
plt.xlabel('time/s');
plt.ylabel('height/mm');

plt.legend(loc = 'upper right')
plt.show()